# Featurestore - Churn Demo
## Data Injestion
Christos Aniftos

In [1]:
from google.api_core import operations_v1
from google.cloud.aiplatform_v1beta1.types import io as io_pb2
from google.cloud.aiplatform_v1beta1.types.feature import Feature
from google.cloud.aiplatform_v1beta1 import FeaturestoreServiceClient
from google.cloud.aiplatform_v1beta1 import FeaturestoreOnlineServingServiceClient
from google.cloud.aiplatform_v1beta1.types import entity_type as entity_type_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore as featurestore_pb2
from google.cloud.aiplatform_v1beta1.types import feature_selector as feature_selector_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_service as featurestore_service_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_online_service as featurestore_online_service_pb2

In [2]:
PROJECT_ID=!gcloud config get-value project # returns default project id 
PROJECT_ID=PROJECT_ID[0]

LOCATION = 'europe-west4'
API_ENDPOINT = LOCATION+"-aiplatform.googleapis.com" 
FEATURESTORE_ID = "telco"
ENTITY="customer"

## Define clients for FS admin and data management

In [3]:
# Create admin_client for CRUD 
admin_client = FeaturestoreServiceClient(
    client_options={"api_endpoint": API_ENDPOINT})


In [4]:
LOC_PATH = admin_client.common_location_path(PROJECT_ID, LOCATION)
FS_PATH = admin_client.featurestore_path(PROJECT_ID, LOCATION, FEATURESTORE_ID)
ENTITY_PATH = admin_client.entity_type_path(PROJECT_ID, LOCATION, FEATURESTORE_ID, ENTITY)
FEATURE_PATH = admin_client.feature_path(PROJECT_ID, LOCATION, FEATURESTORE_ID, ENTITY, '{}')

print("Location: \t", LOC_PATH)
print("Feature Store: \t", FS_PATH)
print("Entity: \t", ENTITY_PATH)
print("Feature: \t",FEATURE_PATH)

Location: 	 projects/myfirstproject-226013/locations/europe-west4
Feature Store: 	 projects/myfirstproject-226013/locations/europe-west4/featurestores/telco
Entity: 	 projects/myfirstproject-226013/locations/europe-west4/featurestores/telco/entityTypes/customer
Feature: 	 projects/myfirstproject-226013/locations/europe-west4/featurestores/telco/entityTypes/customer/features/{}


## Injest

#### prepare aggregated data to injest- this creates a temporary bq table with the features

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

query = """
CREATE OR REPLACE TABLE `{project_id}.telco.churn_features_tmp` 
AS SELECT 
t1.* EXCEPT(TotalCharges, SeniorCitizen),
CAST(t1.SeniorCitizen AS BOOL) SeniorCitizen,
SAFE_CAST(t1.TotalCharges AS FLOAT64) as TotalCharges, 
IF(t1.gender='Male',True,False) male,
IF(t1.gender='Female',True,False) female,
ML.STANDARD_SCALER(t1.MonthlyCharges) OVER() as MonthlyCharges_std,
MonthlyCharges_avg_customer_clustered,
CURRENT_TIMESTAMP() as update_time 
FROM `{project_id}.telco.churn` t1
LEFT JOIN (SELECT 
    AVG(MonthlyCharges) MonthlyCharges_avg_customer_clustered,
    SeniorCitizen,
    Partner,
    PhoneService,
    InternetService,
    DeviceProtection,
    TechSupport,
    StreamingTV,
    StreamingMovies,
    PaymentMethod
FROM `myfirstproject-226013.telco.churn`  Group by 
SeniorCitizen,
Partner,
PhoneService,
InternetService,
DeviceProtection,
TechSupport,
StreamingTV,
StreamingMovies,
PaymentMethod) t2 ON
t1.SeniorCitizen=t2.SeniorCitizen AND 
t1.Partner=t2.Partner AND 
t1.PhoneService=t2.PhoneService AND 
t1.InternetService=t2.InternetService AND 
t1.DeviceProtection=t2.DeviceProtection AND 
t1.TechSupport=t2.TechSupport AND 
t1.StreamingTV=t2.StreamingTV AND 
t1.StreamingMovies=t2.StreamingMovies AND 
t1.PaymentMethod=t2.PaymentMethod
""".format(project_id=PROJECT_ID)
query_job = client.query(query)  # Make an API request.

In [ ]:
query_job.result()

### Supported injest modes

avro_source (google.cloud.aiplatform_v1beta1.types.AvroSource):

bigquery_source (google.cloud.aiplatform_v1beta1.types.BigQuerySource):

csv_source (google.cloud.aiplatform_v1beta1.types.CsvSource):

In [16]:
ENTITY_PATH

'projects/myfirstproject-226013/locations/europe-west4/featurestores/universe/entityTypes/customer'

### Injest from temp table to feature store

In [17]:
import_request = featurestore_service_pb2.ImportFeatureValuesRequest(
    entity_type=ENTITY_PATH,
    bigquery_source=io_pb2.BigQuerySource(
        input_uri="bq://{project_id}.telco.churn_features_tmp".format(project_id=PROJECT_ID)
    ),
    feature_specs=[
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="gender", source_field="gender"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="senior_citizen", source_field="SeniorCitizen"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="partner", source_field="Partner"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="dependents", source_field="Dependents"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="tenure", source_field="tenure"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="phone_service", source_field="PhoneService"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="multiple_lines", source_field="MultipleLines"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="internet_service", source_field="InternetService"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="online_security", source_field="OnlineSecurity"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="online_backup", source_field="OnlineBackup"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="device_protection", source_field="DeviceProtection"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="tech_support", source_field="TechSupport"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="streaming_tv", source_field="StreamingTV"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="streaming_movies", source_field="StreamingMovies"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="contract", source_field="Contract"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="paperless_billing", source_field="PaperlessBilling"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="payment_method", source_field="PaymentMethod"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="monthly_charges", source_field="MonthlyCharges"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="male", source_field="male"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="female", source_field="female"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="monthly_charges_std", source_field="MonthlyCharges_std"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="total_charges", source_field="TotalCharges"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="monthly_charges_avg_customer_clustered", source_field="MonthlyCharges_avg_customer_clustered")
    ],
    entity_id_field="customerID",
    feature_time_field="update_time",
    worker_count=1)

In [18]:
%%time
admin_client.import_feature_values(import_request).result()

CPU times: user 85.9 ms, sys: 33.3 ms, total: 119 ms
Wall time: 6min 33s


imported_entity_count: 7043
imported_feature_value_count: 161978